In [ ]:
from HTML import config as Config
from HTML.dataset import *
from HTML.config import ratio, nominal, ordinal, meaningless
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from HTML import preprocessing as ps
%matplotlib inline
data = pd.read_csv(Config.train_path)
data = ps.preprocessing(data)
data.head()


In [ ]:
data = data.drop(['reservation_status','reservation_status_date', 'adr'], axis=1)
data = pd.concat([data, 
                 pd.get_dummies(data['hotel'], drop_first=True), 
                 pd.get_dummies(data['arrival_date_month'], drop_first=True), 
                 pd.get_dummies(data['market_segment'], drop_first=True),
                 pd.get_dummies(data['distribution_channel'], drop_first=True),
                 pd.get_dummies(data['customer_type'], drop_first=True), 
                 pd.get_dummies(data['deposit_type'], drop_first=True),
                 pd.get_dummies(data['meal'], drop_first=True),
                 pd.get_dummies(data['country'], drop_first=True),
                 pd.get_dummies(data['reserved_room_type'], drop_first=True),
                 pd.get_dummies(data['assigned_room_type'], drop_first = True)
                 ], axis=1)
data = data.drop(['assigned_room_type', 'reserved_room_type', 'meal', 'country', 'hotel','arrival_date_month','market_segment','distribution_channel','customer_type', 'deposit_type'], axis=1)
data = data.drop(['ID', 'agent', 'company'], axis=1)
data.head()

In [ ]:
data.info()

In [ ]:
from sklearn.model_selection import cross_val_score
def CreateBalancedSampleWeights(y_train, largest_class_weight_coef):
    classes = np.unique(y_train)
    classes.sort()
    class_samples = np.bincount(y_train)
    total_samples = class_samples.sum()
    n_classes = len(class_samples)
    weights = total_samples / (n_classes * class_samples * 1.0)
    class_weight_dict = {key : value for (key, value) in zip(classes, weights)}
    class_weight_dict[classes[1]] = class_weight_dict[classes[1]] * largest_class_weight_coef
    sample_weights = [class_weight_dict[y] for y in y_train]

    return sample_weights



In [ ]:
# Fit and train
s = []
x = data[set(data.columns) - {'is_canceled'}]
y = data['is_canceled']
from sklearn.svm import SVC
clf = SVC(kernel = 'rbf', random_state=0)
scores = cross_val_score(clf, x, y, cv=5, scoring='f1', verbose=1, n_jobs=2)
s.append(scores)
print(np.mean(scores))
# Predict
# y_pred = classifier.predict(X_test)

# Computing accuracy
# model_accuracy_results['RandomForest'] = model_accuracy(y_test, y_pred)

In [ ]:
plt.plot([1,5,10,50,100,500], np.mean(np.array(s),axis=1))